# LangChain 实战：汽车销售聊天机器人

## 使用 GPT-4 构造销售话术数据

ChatGPT 分享链接：https://chat.openai.com/share/f3e4b9b0-95fb-4c6a-a3c7-f901dd194c91


使用 ChatGPT 构造销售数据的 Prompt 示例：

```
你是中国顶级的汽车销售，现在培训职场新人，请给出100条实用的销售话术。

每条销售话术以如下格式给出，需要带上序号，例如：
1.
[客户问题]
[销售回答]

```

GPT-4 回复结果：

```
了解了，下面是一些基于汽车销售场景的销售话术示例，希望对你有所帮助：

1.
[客户问题]: 这款车的油耗怎么样？
[销售回答]: 这款车采用了最新的节能技术，可以帮您有效降低油耗，实现经济驾驶。例如，它的平均油耗只有每百公里5.8升，远远低于同级别车型。

2.
[客户问题]: 这车安全吗？
[销售回答]: 绝对安全。这辆车配备了最先进的安全系统，包括多达6个气囊，最新的防抱死系统（ABS）和电子稳定程序（ESP），可以在紧急情况下最大程度地保护您和您的家人。

3.
[客户问题]: 保修期是多久？
[销售回答]: 对于这款车型，我们提供全面的保修服务，包括4年或10万公里的原厂保修，还有5年的道路救援服务。您可以完全放心，任何质量问题我们都会负责到底。

4.
[客户问题]: 我能选配颜色和内饰吗？
[销售回答]: 当然可以。我们提供多种颜色和内饰选项，您可以按照自己的喜好和需求进行个性化定制，我们会尽全力满足您的个性化需求。

5.
[客户问题]: 我现在买车，什么时候能提车？
[销售回答]: 如果选择现车，我们可以在您完成购车手续后立即提供车辆。如果您有特别的定制需求，我们会尽快安排生产，并详细告知您预计的交车时间。

6.
[客户问题]: 贷款买车的手续复杂吗？
[销售回答]: 我们有专业的财务顾问团队，会帮您处理所有的贷款手续，让流程变得简单明了。我们还合作多家金融机构，确保您可以得到最优惠的贷款利率和最适合您的还款计划。

7.
[客户问题]: 你们接受旧车置换吗？
[销售回答]: 当然接受。我们提供旧车评估服务，会根据您的车况给出公正的评估价格，并且旧车置换新车会有一定的优惠政策，我们的目标是让您的换车过程简单且具有成本效益。

8.
[客户问题]: 我不太懂车，你们售后服务如何？
[销售回答]: 我们有专业的售后服务团队，从常规保养到维修都有非常详细的服务规范和流程。此外，我们还提供上门服务和一对一的客户专员跟踪服务，确保您的用车无忧。

9.
[客户问题]: 这款车性能怎么样？
[销售回答]: 这款车的性能非常卓越，无论是加速能力还是操控稳定性都是同级别中的佼佼者。您可以预约试驾，亲身体验它的卓越性能。

10.
[客户问题]：这款车的维修成本高吗？
[销售回答]：这款车的整体维护成本在行业中处于较低水平。它的可靠性和延长的保修期限意味着您将享受到持久的高性能和低维修费用。

```


## 使用 Document Transformers 模块来处理原始数据


将 ChatGPT 生成的结果保存到 [real_estate_sales_data.txt](real_estate_sales_data.txt) 文件中

In [38]:
with open("real_car_sales_data.txt") as f:
    real_car_sales = f.read()

### 使用 CharacterTextSplitter 来进行文本分割

- 基于单字符来进行文本分割（separator）
- 基于字符数来决定文本块长度（chunk_size）

参考示例：

```python
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(        
    separator = "\n\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
    is_separator_regex = False,
)
```


In [39]:
from langchain.text_splitter import CharacterTextSplitter

In [40]:
text_splitter = CharacterTextSplitter(        
    separator = r'\d+\.',
    chunk_size = 100,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = True,
)

In [41]:
docs = text_splitter.create_documents([real_car_sales])

Created a chunk of size 103, which is longer than the specified 100
Created a chunk of size 105, which is longer than the specified 100
Created a chunk of size 104, which is longer than the specified 100
Created a chunk of size 106, which is longer than the specified 100
Created a chunk of size 103, which is longer than the specified 100
Created a chunk of size 106, which is longer than the specified 100
Created a chunk of size 111, which is longer than the specified 100
Created a chunk of size 106, which is longer than the specified 100
Created a chunk of size 105, which is longer than the specified 100
Created a chunk of size 108, which is longer than the specified 100
Created a chunk of size 128, which is longer than the specified 100
Created a chunk of size 110, which is longer than the specified 100
Created a chunk of size 111, which is longer than the specified 100
Created a chunk of size 109, which is longer than the specified 100
Created a chunk of size 123, which is longer tha

In [42]:
docs[0]

Document(page_content='\\d+\\.\n[客户问题]: 这款车的油耗怎么样？\n[销售回答]: 这款车采用了最新的节能技术，可以帮您有效降低油耗，实现经济驾驶。例如，它的平均油耗只有每百公里')

In [43]:
len(docs)

101

### 使用 Faiss 作为向量数据库，持久化存储房产销售 问答对（QA-Pair）

In [44]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
import os
BASE_URL = os.environ.get("BASE_URL")
API_KEY = os.environ.get("API_KEY")
db = FAISS.from_documents(docs, OpenAIEmbeddings(base_url=BASE_URL, api_key=API_KEY))

d:\miniconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [45]:
query = "我担心这款车的燃油效率"

In [46]:
answer_list = db.similarity_search(query)

In [47]:
for ans in answer_list:
    print(ans.page_content + "\n")

[客户问题]: 我比较介意油耗，这款车省油吗？
[销售回答]: 这款车的确是市场上油耗表现优异的车型之一。它采用了先进的发动机技术和轻量化设计，以及优化的空气动力学特性，这些都有助于提高燃油经济性，降低您的油耗成本。

\d+\.
[客户问题]: 这款车的油耗怎么样？
[销售回答]: 这款车采用了最新的节能技术，可以帮您有效降低油耗，实现经济驾驶。例如，它的平均油耗只有每百公里

[客户问题]: 我不想经常去加油站，这款车的油箱容量和续航里程怎样？
[销售回答]: 这款车设计了较大的油箱，一次加满油后在正常驾驶条件下可以行驶一段相对较长的距离。并且车辆的燃油效率非常高，可以帮助您减少加油的频次，同时降低长期的运行成本。

[客户问题]: 我的工作需要经常出差，日常使用里程数很高。
[销售回答]: 我们的车辆燃油效率高，配备耐久性强的零部件，能够满足高里程数的需求，并降低维护成本。



In [48]:
db.save_local("real_car_sale")

### 使用 retriever 从向量数据库中获取结果

#### 使用参数 `k` 指定返回结果数量


In [49]:
# 实例化一个 TopK Retriever
topK_retriever = db.as_retriever(search_kwargs={"k": 5})

In [50]:
topK_retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000266C0D204F0>, search_kwargs={'k': 5})

In [51]:
docs = topK_retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

d:\miniconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[客户问题]: 我比较介意油耗，这款车省油吗？
[销售回答]: 这款车的确是市场上油耗表现优异的车型之一。它采用了先进的发动机技术和轻量化设计，以及优化的空气动力学特性，这些都有助于提高燃油经济性，降低您的油耗成本。

\d+\.
[客户问题]: 这款车的油耗怎么样？
[销售回答]: 这款车采用了最新的节能技术，可以帮您有效降低油耗，实现经济驾驶。例如，它的平均油耗只有每百公里

[客户问题]: 我不想经常去加油站，这款车的油箱容量和续航里程怎样？
[销售回答]: 这款车设计了较大的油箱，一次加满油后在正常驾驶条件下可以行驶一段相对较长的距离。并且车辆的燃油效率非常高，可以帮助您减少加油的频次，同时降低长期的运行成本。

[客户问题]: 我的工作需要经常出差，日常使用里程数很高。
[销售回答]: 我们的车辆燃油效率高，配备耐久性强的零部件，能够满足高里程数的需求，并降低维护成本。

[客户问题]: 我关心车辆的燃油效率，你们提供混合动力版本吗？
[销售回答]: 是的，考虑到燃油经济性和环保因素，我们提供了几种混合动力车型。这些车型结合了燃油发动机和电动机的优势，能够提供优异的燃油效率和更低的排放，同时在驾驶体验上也尽可能接近传统汽车。



In [52]:
docs = topK_retriever.get_relevant_documents("我听说这个品牌的售后服务不是很好？")

In [53]:
for doc in docs:
    print(doc.page_content + "\n")

[客户问题]: 我不太懂车，你们售后服务如何？
[销售回答]: 我们有专业的售后服务团队，从常规保养到维修都有非常详细的服务规范和流程。此外，我们还提供上门服务和一对一的客户专员跟踪服务，确保您的用车无忧。

[客户问题]: 我担心买了车后，配件获取困难，维修不方便。
[销售回答]: 您不用担心这一点。我们有完善的配件供应链，确保任何时候都能快速获取所需的配件。此外，我们的服务中心遍布各地，无论您身处何地，都能享受到便捷的维修服务。

[客户问题]: 我担心新车会有质量问题。
[销售回答]: 我们了解买车是一项重要的投资，因此我们保证每一辆出厂的车都经过了严格的质量控制检查。此外，我们提供全面的质保服务以及满意的售后服务来解决您在使用过程中遇到的任何问题。

[客户问题]: 你们的车辆故障率高吗？
[销售回答]: 我们的品牌一直是业内以低故障率著称。我们采用高标准的制造流程和严格的质量检测，确保每一辆车都能达到最高的可靠性要求。

[客户问题]：这款车的维修成本高吗？
[销售回答]：这款车的整体维护成本在行业中处于较低水平。它的可靠性和延长的保修期限意味着您将享受到持久的高性能和低维修费用。
当然可以，接下来的汽车销售话术如下：



#### 使用 similarity_score_threshold 设置阈值，提升结果的相关性质量

In [54]:
# 实例化一个 similarity_score_threshold Retriever
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.8}
)

In [55]:
docs = retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

[客户问题]: 我比较介意油耗，这款车省油吗？
[销售回答]: 这款车的确是市场上油耗表现优异的车型之一。它采用了先进的发动机技术和轻量化设计，以及优化的空气动力学特性，这些都有助于提高燃油经济性，降低您的油耗成本。

\d+\.
[客户问题]: 这款车的油耗怎么样？
[销售回答]: 这款车采用了最新的节能技术，可以帮您有效降低油耗，实现经济驾驶。例如，它的平均油耗只有每百公里

[客户问题]: 我不想经常去加油站，这款车的油箱容量和续航里程怎样？
[销售回答]: 这款车设计了较大的油箱，一次加满油后在正常驾驶条件下可以行驶一段相对较长的距离。并且车辆的燃油效率非常高，可以帮助您减少加油的频次，同时降低长期的运行成本。

[客户问题]: 我的工作需要经常出差，日常使用里程数很高。
[销售回答]: 我们的车辆燃油效率高，配备耐久性强的零部件，能够满足高里程数的需求，并降低维护成本。



### 提取向量数据库中的`销售回答`

In [56]:
docs = retriever.get_relevant_documents(query)

In [57]:
docs[0].page_content

'[客户问题]: 我比较介意油耗，这款车省油吗？\n[销售回答]: 这款车的确是市场上油耗表现优异的车型之一。它采用了先进的发动机技术和轻量化设计，以及优化的空气动力学特性，这些都有助于提高燃油经济性，降低您的油耗成本。'

In [58]:
docs[0].page_content.split("[销售回答] ")

['[客户问题]: 我比较介意油耗，这款车省油吗？\n[销售回答]: 这款车的确是市场上油耗表现优异的车型之一。它采用了先进的发动机技术和轻量化设计，以及优化的空气动力学特性，这些都有助于提高燃油经济性，降低您的油耗成本。']

In [67]:
ans = docs[0].page_content.split("[销售回答] ")[-1].lstrip('\n')

In [68]:
ans

'[客户问题]: 我比较介意油耗，这款车省油吗？\n[销售回答]: 这款车的确是市场上油耗表现优异的车型之一。它采用了先进的发动机技术和轻量化设计，以及优化的空气动力学特性，这些都有助于提高燃油经济性，降低您的油耗成本。'

#### 尝试各种问题

In [69]:
from typing import List

def sales(query: str, score_threshold: float=0.8) -> List[str]:
    retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": score_threshold})    
    docs = retriever.get_relevant_documents(query)
    ans_list = [doc.page_content.split("[销售回答] ")[-1].lstrip('\n') for doc in docs]

    return ans_list

In [72]:
query = "这个价格对我来说有点高"

print(sales(query))

[]


d:\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


In [73]:
print(sales(query, 0.6))

['[客户问题]：这款车的维修成本高吗？\n[销售回答]：这款车的整体维护成本在行业中处于较低水平。它的可靠性和延长的保修期限意味着您将享受到持久的高性能和低维修费用。\n当然可以，接下来的汽车销售话术如下：', '[客户问题]\n我需要考虑一下。\n[销售回答]\n我完全理解您需要时间来考虑这项重要投资。如果您有任何疑问或需要更多信息，我随时愿意协助。同时，如果您能在今天决定，我们有些特别优惠可以提供。', '[客户问题]: 我的预算有限，保险费用怎么样？\n[销售回答]: 我们车型享有良好的安全评级，这有助于降低保险费用。具体费用还需咨询保险公司，视个人驾驶记录和地区而定。', '[客户问题]: 我对这款车感兴趣，但我想再看看其他品牌的类似车型。\n[销售回答]: 我理解您想要做出明智选择的想法，同时我自信地告诉您，我们的车型在性能、舒适度以及价格价值比方面都是市场上最具竞争力的。如果您有需要，我可以提供一些与其他品牌类似车型的对比资料，帮助您做出决定。']


In [74]:
query = "我听说维修成本很高昂"

print(f"score:0.8 ans: {sales(query)}\n")
print(f"score:0.75 ans: {sales(query, 0.75)}\n")
print(f"score:0.5 ans: {sales(query, 0.5)}\n")

d:\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


score:0.8 ans: []

score:0.75 ans: ['[客户问题]：这款车的维修成本高吗？\n[销售回答]：这款车的整体维护成本在行业中处于较低水平。它的可靠性和延长的保修期限意味着您将享受到持久的高性能和低维修费用。\n当然可以，接下来的汽车销售话术如下：', '[客户问题]: 我担心维护成本会很高。\n[销售回答]: 我们的车型在设计之初就充分考虑了维护便利性，部分零件采用的是通用性高的设计。此外，我们还提供了各种预付维护计划，可以帮助您将未来的维护成本固定下来。', '[客户问题]: 我想了解下这款车型的保养成本。\n[销售回答]: 这款车的保养成本实际上相当合理。我们采用的是高效率和长寿命的零部件，降低了更换频率。而且，我们还提供不同的预付保养套餐，让您可以根据自己的需要选择最合适的保养计划。', '[客户问题]: 我关心的是车辆的保养费用。\n[销售回答]: 我们车型的保养周期长，常规保养费用低，且第一次保养免费，帮您节省开支。']

score:0.5 ans: ['[客户问题]：这款车的维修成本高吗？\n[销售回答]：这款车的整体维护成本在行业中处于较低水平。它的可靠性和延长的保修期限意味着您将享受到持久的高性能和低维修费用。\n当然可以，接下来的汽车销售话术如下：', '[客户问题]: 我担心维护成本会很高。\n[销售回答]: 我们的车型在设计之初就充分考虑了维护便利性，部分零件采用的是通用性高的设计。此外，我们还提供了各种预付维护计划，可以帮助您将未来的维护成本固定下来。', '[客户问题]: 我想了解下这款车型的保养成本。\n[销售回答]: 这款车的保养成本实际上相当合理。我们采用的是高效率和长寿命的零部件，降低了更换频率。而且，我们还提供不同的预付保养套餐，让您可以根据自己的需要选择最合适的保养计划。', '[客户问题]: 我关心的是车辆的保养费用。\n[销售回答]: 我们车型的保养周期长，常规保养费用低，且第一次保养免费，帮您节省开支。']



#### 当向量数据库中没有合适答案时，使用大语言模型能力

In [93]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
import os

BASE_URL = 'https://api.xiaoai.plus/v1'
API_KEY = 'sk-jUuTDR1ErlbHv7y55994E96a8c144dC3BcF6891e599dB48e '

llm = ChatOpenAI(model_name="gpt-4-1106-preview", temperature=0.5, base_url=BASE_URL, api_key=API_KEY)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.7}))

In [94]:
qa_chain({"query": "我担心新车一旦驶出展厅会立即贬值"})

APIConnectionError: Connection error.

In [81]:
qa_chain({"query": "最贵的车是哪一辆?"})

d:\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


APIConnectionError: Connection error.

In [82]:
print(sales("最贵的车是哪一辆?"))

[]


d:\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


## 加载 FAISS 向量数据库已有结果

In [85]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
import os

BASE_URL = 'https://api.xiaoai.plus/v1'
API_KEY = 'sk-jUuTDR1ErlbHv7y55994E96a8c144dC3BcF6891e599dB48e '

db = FAISS.load_local("real_car_sale", OpenAIEmbeddings(base_url=BASE_URL, api_key=API_KEY),
allow_dangerous_deserialization='True')

In [86]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4", temperature=0.5, base_url=BASE_URL, api_key=API_KEY)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

d:\miniconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [87]:
qa_chain({"query": "哪种车是最推荐的？"})

APIConnectionError: Connection error.

In [88]:
# 输出内部 Chain 的日志
qa_chain.combine_documents_chain.verbose = True

In [89]:
qa_chain({"query": "哪种车是最推荐的？"})

APIConnectionError: Connection error.

In [90]:
# 返回向量数据库的检索结果
qa_chain.return_source_documents = True

In [91]:
result = qa_chain({"query": "哪种车是最推荐的？"})

APIConnectionError: Connection error.

In [92]:
result

NameError: name 'result' is not defined